Read Data

In [1]:
import pandas as pd
import os

# Verify file existence
filepath = 'wikisql/train.csv'
if not os.path.exists(filepath):
    raise FileNotFoundError("File train.csv not found")

# Load data safely
data = pd.read_csv(os.path.abspath(filepath))  # Use absolute path


Installations

In [2]:
!pip install transformers
!pip install torch torchvision torchaudio
!pip install tf-keras
!pip install SentencePiece
!pip install datasets
!pip install accelerate

Version checks

In [3]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

!pip list | grep -E 'torch|tensorflow'


/opt/anaconda3/lib/python3.12/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


PyTorch version: 2.6.0
CUDA available: False
tensorflow                        2.19.0
tensorflow_estimator              2.15.0
torch                             2.6.0
torchaudio                        2.6.0
torchvision                       0.21.0


----------
Model Choice: T5-small (seq2seq architecture, ideal for text-to-SQL)

In [4]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments, AutoConfig

# Initialize model
config = AutoConfig.from_pretrained("t5-small", use_torch=True)
model = T5ForConditionalGeneration.from_pretrained(
    't5-small')
    # "t5-small", 
    # config=config,
    # torch_dtype=torch.float32)
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Prepare dataset
data['input'] = "Translate to SQL: " + data['question']
data['target'] = data['sql'].str.replace(r'\s+', ' ', regex=True)  # Clean whitespace


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


-------
Dataset Preprocessing

In [5]:
from datasets import Dataset

dataset = Dataset.from_pandas(data[['input', 'target']])

def preprocess(examples):
    model_inputs = tokenizer(
        examples['input'],
        max_length=128,
        truncation=True,
        padding='max_length'
    )
    
    labels = tokenizer(
        examples['target'],
        max_length=128,
        truncation=True,
        padding='max_length'
    )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess, batched=True)


Map:   0%|          | 0/56355 [00:00<?, ? examples/s]

-------
Training Configuration

In [6]:
training_args = TrainingArguments(
    output_dir="./sql-generator",
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,  # Use separate validation set in practice
)


/opt/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


-----
Model Training

In [7]:
trainer.train()


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.070700,0.048900
2,0.055300,0.037186
3,0.048200,0.030320
4,0.040600,0.025964
5,0.036200,0.024380


TrainOutput(global_step=35225, training_loss=0.05571517603206161, metrics={'train_runtime': 21566.5485, 'train_samples_per_second': 13.065, 'train_steps_per_second': 1.633, 'total_flos': 9533984027443200.0, 'train_loss': 0.05571517603206161, 'epoch': 5.0})

------
Saving the model

In [8]:
trainer.save_model("sql_generator")  # Saves PyTorch model files
tokenizer.save_pretrained("sql_generator")  # Saves tokenizer config

('sql_generator/tokenizer_config.json',
 'sql_generator/special_tokens_map.json',
 'sql_generator/spiece.model',
 'sql_generator/added_tokens.json')

In [16]:
from huggingface_hub import ModelCard, ModelCardData

# Generate model card
card_data = ModelCardData(
    language="en",
    license="mit",
    library_name="transformers",
    tags=["text2text-generation", "WikiSQL"]
)
card = ModelCard.from_template(
    card_data=card_data,
    model_id="nl2sql_spring25",
    task="text2text-generation",
    library_name="transformers",
    datasets="WikiSQL",
    metrics=["exact_match"]
)
card.save("sql_generator/README.md")

# Compress for transfer
import shutil
shutil.make_archive("sql_generator", 'zip', "sql_generator")


'/Users/madridista/Documents/Academics/Spring 25/NLP/Grp Project/Code/nlp-to-sql/sql_generator.zip'

-------
Using the model

In [17]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load from directory
model = T5ForConditionalGeneration.from_pretrained(
    "sql_generator",
    local_files_only=True
)

tokenizer = T5Tokenizer.from_pretrained(
    "sql_generator",
    local_files_only=True
)

# Verify loading
def generate_sql(question):
    inputs = tokenizer(f"Translate to SQL: {question}", return_tensors="pt")
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generate_sql("Show notes for South Australia"))
# Should output: SELECT Notes FROM table WHERE Current slogan = 'SOUTH AUSTRALIA'


SELECT Notes FROM table WHERE Country = south australia


In [ ]:
# def generate_sql(question):
#     input_text = f"Translate to SQL: {question}"
#     inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True)
    
#     outputs = trainer.generate(
#         input_ids=inputs.input_ids,
#         attention_mask=inputs.attention_mask,
#         max_length=128
#     )
    
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

# # Example usage
# print(generate_sql("Show me notes from South Australia"))
# # Output: SELECT Notes FROM table WHERE Current slogan = 'SOUTH AUSTRALIA'
